## Mini Project III

In [1]:
import numpy as np
import plotly.express as px
import matplotlib.pyplot as plt
import pandas as pd

first 8 of acct_nbr are uniform.

acct_nbr contains no duplicates :)

one years worth of transactions

33 states, 69 cities

Investigate clustering by 1st postal code

In [51]:
customer = pd.read_csv('data/twm_customer.csv', delimiter=';')

account = pd.read_csv('data/twm_accounts.csv', delimiter=';')
transactions = pd.read_csv('data/twm_transactions.csv', delimiter=';')

account_checking = pd.read_csv('data/twm_checking_acct.csv', delimiter=';')
account_credit = pd.read_csv('data/twm_credit_acct.csv', delimiter=';')
account_saving = pd.read_csv('data/twm_savings_acct.csv', delimiter=';')

transaction_checking = pd.read_csv('data/twm_checking_tran.csv', delimiter=';')
transaction_credit = pd.read_csv('data/twm_credit_tran.csv', delimiter=';')
transaction_saving = pd.read_csv('data/twm_savings_tran.csv', delimiter=';')

# Cleaning
### First
* strip white spaces from str columns
* convert dates to computable date types, and times to just the hour.
* make account numbers uniform
* drop columns



In [52]:
all_dfs = [customer,account,transactions,account_checking,account_credit,account_saving,transaction_checking,transaction_credit,transaction_saving]

for i in range(len(all_dfs)):
    for col in all_dfs[i].select_dtypes('object'):
        all_dfs[i][col] = all_dfs[i][col].str.strip()

customer,account,transactions,account_checking,account_credit,account_saving,transaction_checking,transaction_credit,transaction_saving = all_dfs

tran_dfs = [transactions,transaction_checking,transaction_credit,transaction_saving]
for i in range(len(tran_dfs)):
    tran_dfs[i]['tran_time'] = tran_dfs[i]['tran_time'].astype(str)
    tran_dfs[i] = tran_dfs[i].replace('',np.nan)
    tran_dfs[i][(tran_dfs[i]['tran_time'].str.len() > 6) | (tran_dfs[i]['tran_time'].str.len() <=4)] = np.nan
    tran_dfs[i]['tran_time'] = pd.to_datetime(tran_dfs[i]['tran_time'], format='%H%M%S').dt.hour
    tran_dfs[i]['tran_date'] = pd.to_datetime(tran_dfs[i]['tran_date'], format='%d.%M.%Y').dt.normalize()

transactions,transaction_checking,transaction_credit,transaction_saving = tran_dfs

account_number_dfs =[account,transactions,account_credit]

for df in account_number_dfs:
    df['acct_nbr'] = df['acct_nbr'].astype(str)

    df['acct_nbr'] = np.where(df['acct_nbr'].str.len() == 16, df['acct_nbr'].str[8:],df['acct_nbr'])

account,transactions,account_credit = account_number_dfs

In [ ]:
customer = customer.drop(['first_name','last_name','street_nbr','street_name'])

,cust_id,income,age,years_with_bank,nbr_children,gender,marital_status,name_prefix,first_name,last_name,street_nbr,street_name,postal_code,city_name,state_code
4,1362548,44554,59,9,2,F,4,,Barbara,O'Malley,6578,C,10138,New York City,NY
14,1362996,23351,51,1,2,F,2,Mrs.,Robin,Johnson,5364,Capital,19136,Philadelphia,PA
21,1362792,3158,80,0,0,M,2,,Eric,Parent,5250,Cactus,10088,New York City,NY
23,1363261,19494,46,6,0,F,1,Ms.,Paula,Wittenberg,5488,Cherry,10098,New York City,NY
25,1363057,6493,74,4,0,F,2,,Carmen,Williamson,7541,Parsons,14667,Rochester,NY
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
718,1362550,0,15,0,0,M,1,,Parvadha,O'Malley,6578,C,10138,New York City,NY
734,1362998,0,17,2,0,F,1,,Sheila,Johnson,5364,Capital,19136,Philadelphia,PA
735,1363242,4431,62,2,0,M,3,Mr.,Jon,Klein,2070,Maple,19133,Philadelphia,PA
736,1363467,29521,48,4,0,F,1,,Linda,Schnack,10613,Vance Road,10105,New York City,NY


In [6]:
customer.city_name.value_counts().count()

69

In [40]:
customer.state_code.value_counts().count()

33

In [7]:
for col in transactions.select_dtypes('object'):
    transactions[col] = transactions[col].str.strip()

In [8]:
account.head()

,acct_nbr,cust_id,acct_type,account_active,acct_start_date,acct_end_date,starting_balance,ending_balance
0,13628063,1362806,SV,Y,10.12.1995,NaN,1430.22,284.58
1,13627090,1362709,CC,Y,15.3.1993,NaN,266.34,496.15
2,13628360,1362836,CC,Y,18.3.1992,NaN,55.90,1000.00
3,13633112,1363311,CK,Y,6.7.1995,NaN,11017.13,968.46
4,13633610,1363361,CC,Y,17.6.1994,NaN,849.37,462.28


In [9]:
account_checking.head()

,cust_id,acct_nbr,minimum_balance,per_check_fee,account_active,acct_start_date,acct_end_date,starting_balance,ending_balance
0,1362548,13625482,3000,0.00,Y,11.11.1986,NaN,6004.34,569.65
1,1362487,13624872,3000,0.00,Y,12.1.1995,NaN,2781.07,1401.21
2,1363017,13630172,3000,0.00,Y,19.3.1994,NaN,2694.91,147.15
3,1362752,13627522,100,0.15,Y,8.1.1994,NaN,487.69,3.76
4,1363282,13632822,100,0.15,Y,5.3.1988,NaN,133.90,84.18


In [10]:
transactions[transactions['tran_id'] == 26]

,tran_id,acct_nbr,tran_amt,principal_amt,interest_amt,new_balance,tran_date,tran_time,channel,tran_code
193,26,13632842,182.12,182.12,0.0,202.77,11.2.1995,233027,A,DP
300,26,13624840,-158.95,-158.95,0.0,-368.11,4.7.1995,135231,E,CG
344,26,13629750,276.93,270.86,6.7,-160.28,1.11.1995,0,M,PM
369,26,13628972,-0.15,-0.15,0.0,147.34,8.8.1995,0,,FK
384,26,13626312,-225.16,-225.16,0.0,1867.99,29.3.1995,170318,E,WD
...,...,...,...,...,...,...,...,...,...,...
77320,26,13626072,0.00,0.00,0.0,63.95,8.1.1995,84632,A,IQ
77337,26,13634580,-98.13,-98.13,0.0,-1100.00,16.7.1995,1206,E,CG
77376,26,13631730,-123.83,-123.83,0.0,-1400.00,4.8.1995,114220,E,CG
77514,26,13632752,-0.15,-0.15,0.0,10.60,11.1.1995,0,,FK


In [11]:
account[account['cust_id'] == 1363017]

,acct_nbr,cust_id,acct_type,account_active,acct_start_date,acct_end_date,starting_balance,ending_balance
766,13630172,1363017,CK,Y,19.3.1994,NaN,2694.91,147.15
949,13630170,1363017,CC,Y,8.2.1995,NaN,0.00,1000.00
1191,13630173,1363017,SV,N,27.4.1994,31.10.1995,2011.02,2901.35
